<a href="https://colab.research.google.com/github/NikFloden/Art-Style-Transfer-Using-Neural-Networks/blob/main/Milestone_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone 4 - Implementing Neural Style Transfer

__Objective__: Use Artificial Intelligence to create art using the Neural Style Transfer algorithm that was introduced by Gatys et al. in their 2015 paper titled, ‘A Neural Algorithm of Artistic Style’. We will be applying the artistic style from an image (we’ll call this the style image) to another image (our content image). We will be performing this style transfer using a 3-component loss function that includes the Content Loss, Style Loss, and the Total Variation Loss

__Notes__:
- Recall from earlier lessons that within the space of a pretrained CNN, there is stored ‘knowledge’ known as the latent space. The filters of pretrained CNNs are hierarchical learners which means the lower layers store information relating to simple local information such as blobs, colors, edges, etc. Mid-tier layers capture a combination of things from lower layers to recognize corners and simple shapes. The upper layers capture more complex patterns and abstract features. 
- It is advised you use Keras instead of the Keras included with TensorFlow 2.0.
- Use the requirements.txt file attached


__Workflow__:

1. Let’s start developing the intuition for our loss functions. We first need to develop a Content Loss function. 
    - In order to preserve the contour lines and spatial layouts of our content image, if we propagate our image through a pretrained CNN and look at the activations of the upper or higher layers, it should activate on well defined/recognizable abstract qualities in the image. 
    - You will need to compute the activation of a specific upper layer, for instance, if using VGG19 (recommended), you can utilize the filter named, `block4_conv2` for both your content and style images.
    - Compute the L2-norm (sum of squared differences) between these activations. The content loss is the L2-norm between the features of our input image and the features of the generated, output image.
    - Our Content Loss function’s aim is to ensure the output generated image will have some similarity to the content image.
2. Let’s develop the intuition behind our Style loss. Instead of a single upper layer, we will utilize multiple layers of our pretrained CNN to obtain our style loss function. This is because we wish to capture multi-scale representations and textures from our style image. This allows us to capture the local style and avoid capturing global arrangements.
    - We use multiple layers for example (if using VGG19) `['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'Block5_conv1']
    - Our goal during training is to minimize the loss between the style of our generated image and the style of our style image. This ensures that the style of our generated images is correlated to the style of the style image. 
    - In order to build this style loss function, we need to compute the correlations between the activation layers of our selected CNN filters. To do this we compute the Gram Matrix between the activations of these layers. A Gram matrix is the inner product of a set of features maps.
3. Build a function that returns the Gram Matrix. The Gram Matrix in our case is the dot product between the input vectors (feature maps) and their transpose. This can be built by:
    - Flattening the input features using `features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))`
    - Then finding the dot product with its transpose. This can be done with the Keras backend function shown here: `gram = K.dot(features, K.transpose(features))`
    - Computing the L2 norm allows us to force our generated output image to have similar style characteristics, but not necessarily the same structural contents, as the style image.
4. We finally get to our third loss function, the Total-Variation loss which now operates only on the output image. This function was not part of the original paper by Gatys et al. but was introduced later on due to better spatial smoothness in the output image (i.e. more locally coherent).  It’s effectively a measure of noise in an image and thus by lowering its loss, we make more aesthetically appealing images. To obtain the Total Variation loss, you will need to:
    - Shift the image one pixel to the right and calculate the sum of squared error between the transferred and original. 
    - To square our tensors we can use the Keras backend function `k.square` and to get the sum we can use `k.sum`
    - Then we do this again by we shift the image one pixel down this time
5. Now you can combine all three loss functions to obtain one single loss function to minimize:
    - This is given as `total_loss = [style(style_image) - style(generated_image)] + [content(original_image) - content(generated_image)] + total_variation_loss`
6. Now that you have loss functions, let’s start putting together some helper utility functions to implement Neural Style Transfer
    - Create a `pre_process_image()` function that takes the image or input path to an image as its argument and outputs an image using the `vgg19.preprocess_input()` function.
    - Create a `deprocess_image()` function that removes the zero-center by mean pixel and clips the output values between 0 and 255
7. Define two Keras variables to store our content and style images to get the tensor representations of our images.
    - Use the Keras backend function `K.variable` to get the tensor representation
    - Create a placeholder for our generated output image using `K.placeholder((1, img_nrows, img_ncols, 3))`
8. Combine the three images into one single input tensor using `K.concatenate()`.
9. Load the pretrained (imagenet)  VGG19 network using our input tensor (i.e. our 3 images) as the input without loading the top of the network.
10. Select your network layer you will be using for your Content Loss.  We use upper layers so that high-level features are represented so selecting `block5_conv2` for instance would be a good choice.  Get the symbolic outputs of each key layer like this:
    - `outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])`
    - Get the single  `layer_features = outputs_dict['block5_conv2']`
11. Now we can extract the features from the layer that we chose from the input tensor like this:
    - `content_image_features = layer_features[0, :, :, :]`
    - `combined_features = layer_features[2, :, :, :]`
12. Finally, we loop through these feature_layers to calculate the style loss.
13. We now need to create a way to get the gradients of the generated image with respect to the loss.
    - You can use Keras’s `K.gradients` and `k.function` to build this.
    - After which create a simple function called `eval_loss_and_grads` that will  return the loss and gradients

14. Create a class called `evaluator` that contains methods that calculate the overall loss and gradients as described previously. This is needed so that we can compute our  loss and gradients in one pass while retrieving them via two separate functions, `loss` and `grads`. This is done because scipy.optimize  requires separate functions for loss and gradients, but computing them separately would be inefficient.

15. All our building blocks are now in place to implement the Neural Style Transfer Algorithm. However, we can fine-tune the weightings of the contribution of the style and content images by using some weighting parameters. These are multiplied by each type of loss. They are the `content_weight`, `total_variation_weight` and the `style_weight`.

16. Start iteratively minimize our total loss function using the scipy-based optimization method called `scipy.optimize.fmin_l_bfgs_b`.
    - To be track on this training process for each iteration print out some logging information such as the Iteration no. and  the loss 
    - Display your Neural Style Transfer generated image the end
17. Experiment with different weighting combinations, different layers, number of iterations and of course changing your content and style images to create amazing AI-generated art using NST!


In [10]:
# Import our modules. Note we use keras and not the TensorFlow2.0 keras
import keras as K
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
import tensorflow as tf
# Import our VGG19 model
from tensorflow.keras.applications import vgg16 as vgg


### Download our style and base images

In [1]:
!wget -O style_image.jpg https://i.imgur.com/GwoGyMl.jpg #Download Features Style Image
#!wget -O style_image.jpg https://i.imgur.com/UkgSWFV.jpg #Download Candy Style Image
#!wget -O style_image.jpg https://i.imgur.com/ivOAEV1.jpg #Download Mosaic Style Image

!wget -O base_image.jpg https://i.imgur.com/UCDA6NR.jpg #Download Base Image



--2022-02-16 03:49:18--  https://i.imgur.com/GwoGyMl.jpg
Resolving i.imgur.com (i.imgur.com)... 199.232.76.193
Connecting to i.imgur.com (i.imgur.com)|199.232.76.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 321995 (314K) [image/jpeg]
Saving to: ‘style_image.jpg’

style_image.jpg     100%[===================>] 314.45K  --.-KB/s    in 0.03s   

2022-02-16 03:49:18 (9.73 MB/s) - ‘style_image.jpg’ saved [321995/321995]

--2022-02-16 03:49:18--  https://i.imgur.com/UCDA6NR.jpg
Resolving i.imgur.com (i.imgur.com)... 199.232.76.193
Connecting to i.imgur.com (i.imgur.com)|199.232.76.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115057 (112K) [image/jpeg]
Saving to: ‘base_image.jpg’

base_image.jpg      100%[===================>] 112.36K  --.-KB/s    in 0.02s   

2022-02-16 03:49:18 (5.36 MB/s) - ‘base_image.jpg’ saved [115057/115057]



In [11]:
# Point to our image paths for our content/base image and style images
base_image = PIL.Image.open("../content/base_image.jpg")
tf.keras.utils.get_file()
style_image = PIL.Image.open("../content/style_image.jpg")


### These are the weights of the different loss components

In [ ]:
# Also, higher total_variation_weight implies higher spatial smoothness.


#### Set the dimensions of the generated image

In [ ]:
# dimensions of the generated picture.


#### Preprocess Utility Function 

In [ ]:
# util function to open, resize and format pictures into appropriate tensors


#### Deprocess Utility Function 

In [ ]:
def deprocess_image(x):
 

#### Convert out images to tensor representations

In [ ]:
# get tensor representations of our images


#### Examine what a tensor looks like

In [ ]:
# Examine our tensor shape


#### Create a blank placehold image to hold our output image

In [ ]:
# this will contain our generated image


#### Combine the 3 images into a single Keras tensor 

In [ ]:
# combine the 3 images into a single Keras tensor


### Load our pretrained VGG19 without the head and the toplayer

In [12]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
base_model = vgg.VGG16(weights='imagenet', 
                       include_top=False, 
                       input_shape=(48, 48, 3))

#### Understand how we get the layer names and how we create a dictionary of layer names as the key and the outputs of layer as the key

In [27]:
# We can extract the model names using the .name method to access it
numlayer = len(base_model.layers)
names = [base_model.layers[i].name for i in range(numlayer)]
# Likewise you can do the same for the model output
outputs = [base_model.layers[i].output for i in range(numlayer)]
name_output_dict = dict(zip(names, outputs))

In [38]:
# Examine what the dictionary stores
print(name_output_dict[])

KerasTensor(type_spec=TensorSpec(shape=(None, 48, 48, 3), dtype=tf.float32, name='input_3'), name='input_3', description="created by layer 'input_3'")


# The Gram Matrix function

In [29]:
# compute the neural style loss
# first we need to define 4 util functions

# the gram matrix of an image tensor (feature-wise outer product)
def gram_matrix(x):
   assert K.ndim(x) == 3
   features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
   gram = K.dot(features, K.transpose(features))
   return gram

# The Style Loss function

In [30]:
# the "style loss" is designed to maintain
# the style of the reference image in the generated image.
# It is based on the gram matrices (which capture style) of
# feature maps from the style reference image
# and from the generated image
def style_loss(style, combination):
   assert K.ndim(style) == 3
   assert K.ndim(combination) == 3
   S = gram_matrix(style)
   C = gram_matrix(combination)
   channels = 3
   size = img_nrows * img_ncols
   return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

# The Content Loss function

In [31]:
# an auxiliary loss function
# designed to maintain the "content" of the
# base image in the generated image
def content_loss(base, combination):
   return K.sum(K.square(combination - base))

# The Total Variation Loss function

In [39]:
# the 3rd loss function, total variation loss,
# designed to keep the generated image locally coherent
def total_variation_loss(x):
   assert K.ndim(x) == 4
   a = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
   b = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
   return K.sum(K.pow(a + b, 1.25))

## Select the Filter we'll be using for our Contentz Loss

In [40]:
# combine these loss functions into a single scalar


TypeError: ignored

## Select the filters you wish to use to build your Gram Matrix required for our Style Loss Function

### We loop through these feature_layers to calculate the style loss

#### We obtain the gradients of the generated image for the respective loss, and then use it to create a tensor function that returns the gradients 

In [ ]:
# get the gradients of the generated image wrt the loss


#### We use this function in the Evaluator class defined below so that we can compute our  loss and gradients in one pass while retrieving them via two separate functions, `loss` and `grads`. This is done because scipy.optimize  requires separate functions for loss and gradients, but computing them separately would be inefficient.

In [ ]:
# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.




# We can now implement our Neural Style Transfer Algorithm!

In [ ]:
import matplotlib.pyplot as plt

# Specify Iterations to run

# Enlarge figure view when displaying final output


# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss



    # save current generated image after deprocessing
 

# Display our images


__Summary__:

In this notebook we:
- Loaded a pretrained (imagenet) VGG19 model without its top layer and used it to implement the Neural Style Transfer Algorithm.
- We created 3 loss function, content loss, style loss, and total variance loss and combined them into a single loss function that was minimized in order to produce a generated image that copied the style of our style image onto our content image. 

__Deliverable__:

The deliverable is a Jupyter Notebook documenting your workflow as you create your loss functions and then combine them using the recommended Keras functions. You are then to load your own content and style images and implement the Neural Style Transfer algorithm to create your own Art. Create a few different variations of your Art using different layers and weighting parameters. 
